# **Integrate Receipt Extraction MCP Server on AgentCore Runtime to AgentCore Gateway**

This MCP server help you extract receipt photo information such as store name, purchase date and total/amount then write output of extract information to Amazon DynamoDB table and send email of output of extract information using Amazon SNS. Then test this MCP server with Langchain.

In [1]:
from google.colab import userdata
import os
# Retrieve AWS credentials from Colab Secrets
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWSACCESSKEY')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWSSECRETKEY')

# **Creating Receipt Extraction MCP Server**

In [2]:
%%writefile mcp_server.py
from fastmcp import FastMCP
from PIL import Image
from google import genai
from pydantic import BaseModel, Field
from google.genai import types
from decimal import Decimal
import boto3
import json
import os
import tempfile

# MCP Server using FastMCP
mcp = FastMCP("Receipt Extraction MCP Server", stateless_http=True, host="0.0.0.0")

region = "us-west-2"
s3 = boto3.client('s3', region_name=region)

# Retrieve Gemini API Key from AWS Secret Manager
secretmanager = boto3.client('secretsmanager', region_name=region)
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
apikey = secret_json["GEMINI_API_KEY"]
geminiclient = genai.Client(api_key=apikey)

# Retrieve AWS Account ID for Amazon SNS TopicARN
sts_client = boto3.client('sts')
response = sts_client.get_caller_identity()
aws_account_id = response['Account']

@mcp.tool()
def receiptExtraction(inputphoto: str):
    class ReceiptExtractionResult(BaseModel):
        """Extracted receipt information."""
        storeName: str = Field(description="Name of store or store name. Must uppercase.")
        purchaseDate: str = Field(description="Purchase date with \"DD-MM-YYYY\" format date.")
        total: float = Field(description="Total or amount. Number and (.) only without any such as $ or other currency.")

    file_name, file_extension = os.path.splitext(inputphoto)
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_photo_file:
        local_file_path = temp_photo_file.name
        s3.download_file("receipts-extraction", inputphoto, local_file_path)

    image = Image.open(local_file_path)
    prompts = """
    Extract store name, purchase date and total/amount based this receipt photo with structure output based ReceiptExtractionResult class format.
    """
    gemini_response = geminiclient.models.generate_content(
        model="gemini-2.5-flash",
        contents=[image, prompts],
        config={
            "response_mime_type": "application/json",
            "response_json_schema": ReceiptExtractionResult.model_json_schema(),
        },
    )
    response_parsed = ReceiptExtractionResult.model_validate_json(gemini_response.text)
    os.remove(local_file_path)
    s3.delete_object(Bucket="receipts-extraction", Key=inputphoto)
    return {"storeName": response_parsed.storeName, "date": response_parsed.purchaseDate, "total": response_parsed.total}

@mcp.tool()
def writeOutput(storeName: str, purchase_date: str, total: float):
    """Write receipt extraction results to Amazon DynamoDB"""
    dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
    table_name = "receiptsExtraction"
    table = dynamodb.Table(table_name)
    table.put_item(Item={
        'storeName': storeName,
        'date': purchase_date,
        'total': Decimal(str(total))
    })
    return {"message": f"Check your {table_name} table in Amazon DynamoDB"}

@mcp.tool()
def sendEmail(storeName: str, purchase_date: str, total: float):
    """Send email notification with receipt extraction results using Amazon SNS"""
    sns = boto3.client('sns', region_name="us-west-2")
    sns.publish(
        TopicArn=f"arn:aws:sns:us-west-2:{aws_account_id}:receiptsExtractionEmail",
        Message=f"Output of this receipt photo:\n\nStore Name: {storeName}\n\nPurchase Date: {purchase_date}\n\nGrand Total: {total}",
        Subject=f"Result of Receipt Extraction - {storeName}"
    )
    return {"message": f"Check your email with subject Result of Receipt Extraction - {storeName}"}

mcp.run(transport="streamable-http")

Writing mcp_server.py


In [3]:
%%writefile requirements.txt
mcp
fastmcp
bedrock-agentcore
boto3
google-genai
langchain[google-genai]
pydantic
Pillow

Writing requirements.txt


In [ ]:
# Install all libraries
!pip install -r requirements.txt bedrock-agentcore-starter-toolkit langchain-mcp-adapters

# **Create Amazon Cognito User Pool, Domain, Resource Server and User Pool Client for AgentCore Gateway Inbound Authentication**

In [5]:
import boto3
region = "us-west-2"

# Initialize Cognito client
cognito = boto3.client('cognito-idp', region)

# Create Cognito User Pool
user_pool_response = cognito.create_user_pool(
    PoolName = "MCP Server to Gateway Pool"
)
cognito_pool_id = user_pool_response['UserPool']['Id']
user_pool_id_without_underscore = cognito_pool_id.replace("_", "").lower()
cognito.create_user_pool_domain(
    Domain = user_pool_id_without_underscore,
    UserPoolId = cognito_pool_id
)

# Create Cognito Resource Server for User Pool
cognito.create_resource_server(
    UserPoolId = cognito_pool_id,
    Name = "GatewayResourceServer",
    Identifier = "gateway-resource-server",
    Scopes = [
        {"ScopeName": "read", "ScopeDescription": "Read access"},
        {"ScopeName": "write", "ScopeDescription": "Write access"}
    ]
)

# Create Cognito User Pool Client
app_client_response = cognito.create_user_pool_client(
    UserPoolId = cognito_pool_id,
    ClientName = "MCP Server to Gateway Pool Client",
    GenerateSecret = True,
    AllowedOAuthFlows = ['client_credentials'],
    AllowedOAuthScopes = ["gateway-resource-server/read", "gateway-resource-server/write"],
    AllowedOAuthFlowsUserPoolClient = True,
    SupportedIdentityProviders = ["COGNITO"],
    ExplicitAuthFlows = [
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']
cognito_client_secret = app_client_response['UserPoolClient']['ClientSecret']

# **Create Amazon Cognito User Pool, Domain, Resource Server and User Pool Client for AgentCore Runtime Inbound Authentication**

In [6]:
import boto3
region = "us-west-2"

# Initialize Cognito client
cognito = boto3.client('cognito-idp', region)

# Create Cognito User Pool
user_pool_response = cognito.create_user_pool(
    PoolName = "Gateway to Receipt Extraction MCP Server Pool"
)
runtime_cognito_pool_id = user_pool_response['UserPool']['Id']
user_pool_id_without_underscore = runtime_cognito_pool_id.replace("_", "").lower()
cognito.create_user_pool_domain(
    Domain = user_pool_id_without_underscore,
    UserPoolId = runtime_cognito_pool_id
)

# Create Cognito Resource Server for User Pool
cognito.create_resource_server(
    UserPoolId = runtime_cognito_pool_id,
    Name = "RuntimeResourceServer",
    Identifier = "runtime-resource-server",
    Scopes = [
        {"ScopeName": "read", "ScopeDescription": "Read access"},
        {"ScopeName": "write", "ScopeDescription": "Write access"}
    ]
)

# Create Cognito User Pool Client
app_client_response = cognito.create_user_pool_client(
    UserPoolId = runtime_cognito_pool_id,
    ClientName = "Gateway to Receipt Extraction MCP Server Pool Client",
    GenerateSecret = True,
    AllowedOAuthFlows = ['client_credentials'],
    AllowedOAuthScopes = ["runtime-resource-server/read", "runtime-resource-server/write"],
    AllowedOAuthFlowsUserPoolClient = True,
    SupportedIdentityProviders = ["COGNITO"],
    ExplicitAuthFlows = [
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)

runtime_cognito_client_id = app_client_response['UserPoolClient']['ClientId']
runtime_cognito_client_secret = app_client_response['UserPoolClient']['ClientSecret']

# **Amazon Bedrock AgentCore Runtime Configuration with MCP Protocol and Auth Configuration from Amazon Cognito Runtime User Pool**

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
region="us-west-2"
agent_name="gemini_mcp_server"

runtime = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    protocol="MCP",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [runtime_cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{runtime_cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
runtime

In [ ]:
launch_result = agentcore_runtime.launch()

In [9]:
agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id
encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'

# **Create AgentCore Identity for Runtime Outbound Auth**

In [10]:
identity = boto3.client('bedrock-agentcore-control', region)

cognito_provider = identity.create_oauth2_credential_provider(
    name = "identity-mcp-server",
    credentialProviderVendor = "CustomOauth2",
    oauth2ProviderConfigInput = {
        'customOauth2ProviderConfig': {
            'oauthDiscovery': {
                'discoveryUrl': f"https://cognito-idp.{region}.amazonaws.com/{runtime_cognito_pool_id}/.well-known/openid-configuration",
            },
            'clientId': runtime_cognito_client_id,
            'clientSecret': runtime_cognito_client_secret
        }
    }
)
cognito_provider_arn = cognito_provider['credentialProviderArn']
cognito_provider_name = cognito_provider['name']

# **Create AgentCore Gateway using Gateway User Pool**

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
gateway = GatewayClient(region)

create_gateway = gateway.create_mcp_gateway(
    name = "gateway-mcp-server",
    authorizer_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration"
        }
    },
    enable_semantic_search = False
)

gateway_id = create_gateway["gatewayId"]
gateway_url = create_gateway["gatewayUrl"]

# **Create MCP Server on AgentCore Runtime as a AgentCore Gateway Target**

In [12]:
gateway_target = boto3.client('bedrock-agentcore-control', region)
create_gateway_target_response = gateway_target.create_gateway_target(
    name = "mcp-server-target",
    gatewayIdentifier = gateway_id,
    targetConfiguration = {
        'mcp': {
            'mcpServer': {
                'endpoint': agent_url
            }
        }
    },
    credentialProviderConfigurations = [
        {
            'credentialProviderType': 'OAUTH',
            'credentialProvider': {
                'oauthCredentialProvider': {
                    'providerArn': cognito_provider_arn,
                    'scopes': []
                }
            }
        }
    ]
)

# **Request access token from Amazon Cognito for Inbound Auth**

In [13]:
import requests
user_pool_id_without_underscore = cognito_pool_id.replace("_", "").lower()

url = f"https://{user_pool_id_without_underscore}.auth.{region}.amazoncognito.com/oauth2/token"
headers = {"Content-Type": "application/x-www-form-urlencoded"}
data = {
    "grant_type": "client_credentials",
    "client_id": cognito_client_id,
    "client_secret": cognito_client_secret
}
response = requests.post(url, headers=headers, data=data)
access_token = response.json()["access_token"]

# **Invoke MCP Server using Langchain MCP Client with several sample photo file**

In [14]:
from google import genai
from langchain.chat_models import init_chat_model
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.messages import ToolMessage
from langchain.agents import create_agent
import boto3
import json
import logging

region = "us-west-2"
s3 = boto3.client('s3', region)

# Retrieve Gemini API Key from AWS Secret Manager
secretmanager = boto3.client('secretsmanager', region)
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
api_key = secret_json["GEMINI_API_KEY"]
llm = init_chat_model("google_genai:gemini-2.5-flash", google_api_key=api_key)

def upload_to_s3(file_path):
    s3.upload_file(file_path, "receipts-extraction", file_path)
    return file_path

In [15]:
async def invokemcp(photo):
    # Suppress warning message
    logging.getLogger("mcp.client.streamable_http").setLevel(logging.ERROR)

    print("Connecting to Receipt Extraction MCP Server...")

    # Create MCP client configuration
    client = MultiServerMCPClient(
        {
            "receipt_server": {
                "url": gateway_url,
                "headers": {
                    "Authorization": f"Bearer {access_token}",
                    "Content-Type": "application/json"
                },
                "transport": "http"
            }
        }
    )

    receipt = upload_to_s3(photo)

    # Get MCP server tool
    tools = await client.get_tools()
    print(f"\nAvailable tools: {[tool.name for tool in tools]}")

    # Create Langchain Agent with MCP server tool
    agent = create_agent(llm, tools)

    # Invoke agent with proper context
    print(f"\nProcessing receipt photo file: {receipt}")
    response = await agent.ainvoke({
        "messages": [
            f"Please extract the receipt information from {receipt}, write the output to DynamoDB, and send an email notification."
        ]
    })
    return response

In [16]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/chagee.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['mcp-server-target___receiptExtraction', 'mcp-server-target___sendEmail', 'mcp-server-target___writeOutput']

Processing receipt photo file: photos/chagee.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/chagee.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='b3dba361-16fe-409a-a635-10b46e7df959'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'mcp-server-target___receiptExtraction', 'arguments': '{"inputphoto": "photos/chagee.jpg"}'}, '__gemini_function_call_thought_signatures__': {'4772451b-567f-4017-a411-1f36a0357e0a': 'Cs0FAXLI2nwmEe1xPam11W5+XM02WRyfE4F4pwMGX3lLY5XKA/5o8EqXgbl7iv63b4764WUgU6ncjzUchPSAGDvp3LNPdqB0Lw+f0R1mvNlqHGngA15FQk4yCRlVrHYVfxHVRgAzQaIz/wbE5zs8UpkmbHQIMN3bVCAVWlUJ0Cob6EtUIchr6Rr5ozp0Om9aH6Z8GsnxYhzntf/G9N27CEY46m4tT+TmEgPiNq5irMkRqWuHYr5WJGp7is12Q4xWGazdfYshyHy+9cnDcj

In [17]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'CHAGEE CHANGI CITY POINT', 'date': '10-12-2023', 'total': 7.3}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - CHAGEE CHANGI CITY POINT'}


In [18]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/playmade.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['mcp-server-target___receiptExtraction', 'mcp-server-target___sendEmail', 'mcp-server-target___writeOutput']

Processing receipt photo file: photos/playmade.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/playmade.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='e3a3f660-b871-406f-9356-e2b28c50f9cc'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'mcp-server-target___receiptExtraction', 'arguments': '{"inputphoto": "photos/playmade.jpg"}'}, '__gemini_function_call_thought_signatures__': {'31877817-327e-45cd-b200-2d9663d7ada2': 'CtgEAXLI2nwhm33NTo04N2nKksQ1A/JbFkv/sEd2gRGnx0uDcYBsY5FmsBn0c6PsfaCxEGXxWIYdZdpRXdm9G/wizktVa5pHGlz/XuC4umhQH5s5cRm9mZVzsdVKWZ2HNZnbr6F2pCO3crE/F68SAdPc0SPiWncBFI922jg5mfiflgD/GV1nTxnc/FJkoEqYB2WuF0qSjVhgEZdnMd88wQNFlm5zUva7X/Jh6J5yywOiJy+K4jp7UOqzgoRkaSO8gK0LGFCDF4KS

In [19]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'PLAYMADE', 'date': '19-01-2024', 'total': 7.0}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - PLAYMADE'}


In [20]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/strongflour.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['mcp-server-target___receiptExtraction', 'mcp-server-target___sendEmail', 'mcp-server-target___writeOutput']

Processing receipt photo file: photos/strongflour.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/strongflour.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='d97dab47-fb84-4f5d-8fc5-e7ae7ddc3087'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'mcp-server-target___receiptExtraction', 'arguments': '{"inputphoto": "photos/strongflour.jpg"}'}, '__gemini_function_call_thought_signatures__': {'653e032e-21a3-44e3-8f97-8a8cdb219c8b': 'CuMCAXLI2nzE13U/DbMHiSFemqphHtr6zyqBSaLuK0gAInzDHb/WBN+iYk8jLoRGnSxG+viPLB9WY9C4IAIwqEdkPcDWTbpNKbNoSVtlTjuTEurG+2RoFu7rG3JMdxardN9wusnbZZshenJCJXUua5CWpDSPq/KUGcqK6QUxyV7yeFtiDgR4sYDe1EdTYigLGwWMSlfeAtcnKdOJWzZnx+i8gdu9EjKbnDTEdDCHhO2fk/gKCi6hlaNfZ9D/1YAO0cO

In [21]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'STRONG FLOUR', 'date': '27-05-2018', 'total': 77.05}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - STRONG FLOUR'}
